# Day 1 - 데이터 수집 및 로드 / 필요 패키지 설치

# 패키지 설치

In [ ]:
install.packages("dplyr")
install.packages("descr")
install.packages("DT")
install.packages("ggplot2")
install.packages("ISLR")
install.packages("MASS")
install.packages("glmnet")
install.packages("randomForest")
install.packages("rpart")
install.packages("ROCR")

# 라이브러리 로드

In [ ]:
## MOUNTING LIBRARY
library(dplyr)
library(descr)
library(DT)
library(ggplot2)
library(ISLR)
library(MASS)
library(glmnet)
library(randomForest)
library(rpart)
library(ROCR)

# 데이터 패스 지정 / 로드

In [ ]:
setwd("E:/Jupyter/datasets")
getwd()

In [ ]:
mushrooms <- read.csv("mushrooms.csv", header = T)

 
# 데이터 정보

<div style="width: 800px;">
    

| 변수명 | 변수설명
| :----:| ----:
|class)|**식용가능** = e, **독성** = p
|cap-shape|bell = b, conical = c, convex = x, flat = f, knobbed = k, sunken = s
|cap-surface|fibrous = f, grooves = g, scaly = y, smooth = s
|cap-color|brown = n, buff = b, cinnamon = c, gray = g, green = r, pink = p, purple = u, red = e, white = w, yellow = y
|bruises|bruises = t, no = f
|odor|almond = a, anise = l, creosote = c, fishy = y, foul = f, musty = m, none = n, pungent = p, spicy = s
|gill-attachment|attached = a, descending = d, free = f, notched = n
|gill-spacing|close = c, crowded = w, distant = d
|gill-size|broad = b, narrow = n
|gill-color|black = k, brown = n, buff = b, chocolate = h, gray = g, green = r, orange = o, pink = p, purple = u, red = e, white = w, yellow = y
|stalk-shape|enlarging = e, tapering = t
|stalk-root|bulbous = b, club = c, cup = u, equal = e, rhizomorphs = z, rooted = r, missing = ?
|stalk-surface-above-ring|fibrous = f, scaly = y, silky = k, smooth = s
|stalk-surface-below-ring|fibrous = f, scaly = y, silky = k, smooth = s
|stalk-color-above-ring|brown = n, buff = b, cinnamon = c, gray = g, orange = o, pink = p, red = e, white = w, yellow = y
|stalk-color-below-ring|brown = n, buff = b, cinnamon = c, gray = g, orange = o,pink = p, red = e, white = w, yellow = y
|veil-type|partial = p, universal = u
|veil-color|brown = n, orange = o, white = w, yellow = y
|ring-number|none = n, one = o, two = t
|ring-type|cobwebby = c, evanescent = e, flaring = f, large = l, none = n, pendant = p, sheathing = s, zone = z
|spore-print-color|black = k, brown = n, buff = b, chocolate = h, green = r, orange =o, purple = u, white = w, yellow = y
|population|abundant = a, clustered = c, numerous = n, scattered = s, several = v, solitary = y
|habitat|grasses = g, leaves = l, meadows = m, paths = p, urban = u, waste = w, woods = d


 </div>

# 데이터 확인

In [ ]:
DT::datatable(mushrooms)

In [ ]:
CrossTable(mushrooms$class)

##### 로드된 데이터중: 식용가능 버섯 비율: 0.518 / 독버섯 비율 : 0.482
# 　

In [ ]:
descr::CrossTable(mushrooms$stalk.root)

In [ ]:
mushrooms %>%
ggplot(aes(stalk.root)) + geom_bar()

# Day 2 - Preprocessing

In [ ]:
mushrooms <- mushrooms[, -17]

###### 모든 값이 'w' 인 무의미한 변수 제거

In [ ]:
str(mushrooms)

In [ ]:
summary(mushrooms)

# Day 3 - 탐색적 데이터 분석

In [ ]:
# 식용/독성 버섯뚜껑 색상
mushrooms %>%
group_by(class) %>%
ggplot(aes(cap.color, fill = class)) + geom_bar(position = "dodge")

In [ ]:
# 식용/독성 버섯 냄새

mushrooms %>%
group_by(class) %>%
ggplot(aes(odor, fill = class)) + geom_bar(position = "dodge")

In [ ]:
mosaicplot( ~ cap.color + class, data = mushrooms, color=T,cex=1.2)

# Day 4 - 데이터 분할
#### 데이터를 트레이닝/검증/테스트데이터 비율로 나누기

In [ ]:
n <- nrow(mushrooms)
idx <- 1:n 

In [ ]:
training.idx <- sample(idx, n * .60) # 전체 데이터에서 60% 만큼을 트레이닝 데이터로 사용

In [ ]:
idx <- setdiff(idx, training.idx) 

In [ ]:
validation.idx <- sample(idx, n * .20)  # 20%를 검증 데이터로
test.idx <- setdiff(idx, validation.idx) # 나머지 값을 테스트 데이터로

In [ ]:
length(training.idx)  # 트레이닝 데이터 개수

In [ ]:
length(validation.idx)  # 검증 데이터 개수

In [ ]:
length(test.idx) # 테스트 데이터 개수

In [ ]:
training <- mushrooms[training.idx, ]
validation <- mushrooms[validation.idx, ]
test <- mushrooms[test.idx, ]

#### 모델만들기

In [ ]:
mushrooms_rf <- randomForest(class ~ . , training)
mushrooms_rf

In [ ]:
importance(mushrooms_rf)

In [ ]:
varImpPlot(mushrooms_rf) # 영향력이 높은 주요 변수들을 확인

# Day 5 - 분류 분석

In [ ]:
head(test, 10) # 테스트 데이터 세트

In [ ]:
result <- predict(mushrooms_rf, newdata = test, type = "prob")[, 'e']

#### 결과 / 식용: 1, 독성: 0

In [ ]:
head(result, 10)

# 데이터 시각화

#### 냄새 특징으로 구분하는 식용/독성 버섯

In [ ]:
disp <- ggplot(mushrooms, aes(class, odor)) 
disp + geom_jitter(aes(color = class)) + scale_color_manual(values=c("green", "red"))

#### 포자 색상으로 구분하는 식용/독성 버섯

In [ ]:
asd <- ggplot(mushrooms, aes(class, spore.print.color))
asd + geom_jitter(aes(color = spore.print.color)) + scale_color_manual(values=c("khaki1","chocolate","black","saddlebrown","orange","green","purple","white","yellow"))

# Day 6 - 분석결과 정리
수집한 데이터셋을 한정으로 하여 분석한 결과,
버섯을 구분하는데 가장 중요한 요소는 냄새와 포자의 색상이고,

예를 들자면 버섯에서 아래와 같은 냄새가 나면 대부분 독이 있는 버섯이다.


- foul 비린내 
- spicy 매운 
- pungent 자극적인 
- musty 곰팡이 
- foul 악취 
- creosote 목재

반면 anise(풀) 냄새나 almond(아몬드) 냄새가 난다면 식용 가능한 버섯으로 추측할 수 있다.

다른 여러 영향력이 높은 변수들과 함께 분석하여 더 정확한 결과를 얻어낼 수 있다.